In [ ]:
import random
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from dotenv import load_dotenv
import os
import requests

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Token y chat ID
TOKEN_TELEGRAM = os.getenv("TOKEN_TELEGRAM")
CHAT_ID_TELEGRAM = os.getenv("CHAT_ID_TELEGRAM")

if not TOKEN_TELEGRAM or not CHAT_ID_TELEGRAM:
    raise ValueError("TOKEN_TELEGRAM o CHAT_ID_TELEGRAM no están configurados en el archivo .env")

# Diccionario con las marcas y sus IDs específicas en la web
MARCAS = {
    "Adidas": 14,
    "Nike": 37,
    "Carharrt": 223,
    "The North Face": 101,
    "Lacoste": 209,
    "Tommy Hilfiger": 119,
    "Ralph Lauren": 91
}

# URLs base con marcadores para el ID de la marca
URLS_BASE = [
    "https://www.vinted.es/catalog?time=1731359532&search_id=18544663024&brand_ids[]={marca_id}&order=newest_first&catalog[]=1206&page=1",
    "https://www.vinted.es/catalog?time=1731359542&search_id=18544663024&brand_ids[]={marca_id}&order=newest_first&catalog[]=79&page=1",
    "https://www.vinted.es/catalog?time=1731359562&search_id=18544663024&brand_ids[]={marca_id}&order=newest_first&catalog[]=34&page=1",
    "https://www.vinted.es/catalog?time=1731359587&search_id=18544663024&brand_ids[]={marca_id}&order=newest_first&catalog[]=581&page=1",
    "https://www.vinted.es/catalog?time=1731359599&search_id=18544663024&brand_ids[]={marca_id}&order=newest_first&catalog[]=582&page=1",
    "https://www.vinted.es/catalog?time=1731359611&search_id=18544663024&brand_ids[]={marca_id}&order=newest_first&catalog[]=583&page=1",
    "https://www.vinted.es/catalog?time=1731359632&search_id=18544663024&brand_ids[]={marca_id}&order=newest_first&catalog[]=585&page=1"
]

# Configuración de Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
service = Service('C:\\Users\\USUARIO\\Desktop\\BOT VINTED\\chromedriver.exe')

# Archivo para almacenar los enlaces ya enviados
ENLACES_ENVIADOS_FILE = 'enlaces_enviados.txt'

# Cargar los enlaces ya enviados desde el archivo
def cargar_enlaces_enviados():
    if os.path.exists(ENLACES_ENVIADOS_FILE):
        with open(ENLACES_ENVIADOS_FILE, 'r') as file:
            return set(file.read().splitlines())
    return set()

# Guardar un nuevo enlace en el archivo
def guardar_enlace_enviado(enlace):
    with open(ENLACES_ENVIADOS_FILE, 'a') as file:
        file.write(enlace + '\n')

def enviar_mensaje_telegram(token, chat_id, mensaje):
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    params = {
        "chat_id": chat_id,
        "text": mensaje
    }
    try:
        response = requests.post(url, params=params)
        response.raise_for_status()
        print("Mensaje enviado correctamente")
    except requests.exceptions.RequestException as e:
        print(f"Error al enviar mensaje: {e}")

def buscar_articulos(marca, marca_id):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    enlaces_enviados = cargar_enlaces_enviados()

    for url_base in URLS_BASE:
        url = url_base.format(marca_id=marca_id)
        print(f"Revisando la URL: {url} para la marca: {marca}")

        try:
            driver.get(url)
            time.sleep(random.randint(5, 10))  # Pausa aleatoria para simular tiempo de carga

            # Desplazamiento aleatorio para simular comportamiento humano
            for _ in range(3):  # Se desplaza la página 3 veces de forma aleatoria
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(random.randint(2, 5))

            items = WebDriverWait(driver, 40).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'new-item-box__overlay')))

            for item in items:
                try:
                    enlace = item.get_attribute('href')
                    nombre = item.get_attribute('title').split(',')[0]
                    precio = item.get_attribute('title').split('precio: ')[1].split(',')[0]

                    if enlace not in enlaces_enviados:
                        try:
                            precio_float = float(precio.replace('€', '').replace(',', '.').strip())
                            if 1 <= precio_float <= 10:
                                mensaje = f"¡Nuevo artículo encontrado!\nMarca: {marca}\nPrecio: {precio}\nEnlace: {enlace}"
                                print(f"Enviando mensaje a Telegram: {mensaje}")
                                enviar_mensaje_telegram(TOKEN_TELEGRAM, CHAT_ID_TELEGRAM, mensaje)
                                enlaces_enviados.add(enlace)
                                guardar_enlace_enviado(enlace)
                                # Pausa aleatoria tras enviar un mensaje para evitar patrones repetitivos
                                time.sleep(random.randint(5, 15))
                        except ValueError as e:
                            print(f"Error al convertir el precio: {e}")
                            continue
                    else:
                        print(f"Enlace duplicado, no se envía: {enlace}")

                except NoSuchElementException:
                    continue

        except TimeoutException:
            print(f"No se pudieron cargar los artículos en la URL: {url} a tiempo.")

    driver.quit()

def filtrar_y_enviar():
    for marca, marca_id in MARCAS.items():
        buscar_articulos(marca, marca_id)

if __name__ == "__main__":
    while True:
        print("Buscando artículos...")
        filtrar_y_enviar()
        # Pausa aleatoria entre iteraciones para evitar detecciones por patrones
        espera = random.randint(120, 600)
        print(f"Esperando {espera // 60} minutos antes de la siguiente búsqueda...")
        time.sleep(espera)


Buscando artículos...


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
